In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import os
import requests
import json

In [3]:
url = 'https://www.collegevine.com/schools/hub/all'
driver = webdriver.Safari()
driver.get(url)

source = driver.page_source
driver.close()

In [4]:
url_list = []
for elem in source[source.index("ListItem"):].split('name')[1:]:
    elem.split(',')
    try:
        college_url = elem[elem.index('https'):elem.index('image')]
        bad_chars = ',"'
     
        college_url = ''.join(map(lambda x: x if x not in bad_chars else '', college_url))
        url_list.append(college_url)
    except:
        pass

In [5]:
url_list.remove('')
url_list.append('https://www.collegevine.com/schools/hub/all/d/holy-names-university')
url_list.sort()

In [65]:
colleges_dict = {}
geo_locater = 'http://www.mapquestapi.com/geocoding/v1/address'

i = 5
for url in url_list[2682:]:
    # getting the name of the College
    name = url[url.index('schools')+len('schools')+1:].split('-')
    for j in range(len(name)):
        name[j] = name[j].capitalize()
    name = ' '.join(name)

    # getting the HTML source
    driver = webdriver.Safari()
    driver.get(url)
    main = driver.page_source
    source = main
    try:
        source = main[main.index('Your estimated net cost'):main.index('Application')]
    except:
        pass
    driver.close()

    # getting the tuitions
    in_state = '>n/a<'
    out_state = '>n/a<'
    try:
        source = source[source.index('In-state'):]
        source = ''.join(source).split('state')
        
        in_state, out_state = source[1], source[2]
        out_state = out_state.split('div')[2]
        in_state = in_state.split('div')[2]
    except:
        pass

    # getting diversity info
    first_year = '>n/a<'
    sex = '>n/a<'
    try:
        source = main[main.index('Enrolled students'):main.index('t--academics')]
        source = source.split('<div')
        first_year = source[3]
        try:
            sex = source[5]
        except:
            pass
    except:
        pass
    p_women = '>n/a<'
    p_men = '>n/a<'
    if 'Enrolled women:' in ''.join(source) and 'Enrolled men:' in ''.join(source):
        p_women, p_men = source[9], source[12]

    retention_rate = '>n/a<'
    try:
        source = '&_&'.join(source)
        source = source[source.index('Retention'):].split('&_&')
        retention_rate = source[3]
    except:
        pass

    international = '>n/a<'
    try:
        source = '&_&'.join(source)
        source = source[source.index('International'):].split('&_&')
        international = source[3]
    except:
        pass

    # Adding the current information so that everything is in the order I want
    temp = {
            'Address': None,
            'Geo Location': None,
            'in state': in_state[in_state.index('>')+1:in_state.index('<')], 
            'out of state': out_state[out_state.index('>')+1:out_state.index('<')],
            'First Year Students': 'n/a',
            'Acceptance Rate (T/F/M)': 'n/a',
            'Total Applicants': 'n/a',
            'Applicants Stats (F/M)': 'n/a',
            'Percent Accepted that Attend': 'n/a',
            'Sex': sex[sex.index('>')+1:sex.index('<')],
            'Percent (F/M)': str( (p_women[p_women.index('>')+1:p_women.index('<')],p_men[p_men.index('>')+1:p_men.index('<')] )),
            'Retention': retention_rate[retention_rate.index('>')+1:retention_rate.index('<')],
            'International Students': international[international.index('>')+1:international.index('<')]
           }

    try:
        temp['First Year Students'] = first_year[first_year.index('>')+1:first_year.index('<')]
    except:
        pass
    # Ethnicity information - Has to be after temp declaration, AND relies on source from 'International'
    try:
        source = source = '&_&'.join(source)
        source = source[source.index('Asian'):].split('&_&')
        for iterator in range(6):
            x = source[0].index(':')
            ethnicity, percent = source[0][:x], source[0][x+1:source[0].index('<')]
            source = source[3:]
            temp[ethnicity.lstrip(' class="ml-2">')] = percent.strip()
    except:
        pass
    
    # Getting address and Lat,Lng
    try: 
        source = main[main.index('Location'):main.index('Campus culture')]
        source = source.split('<div')[6:8]
    
        parameters = {
            'key': '9Lg84ock7HirwVsMFMN3IYWsK4vvs64i',
            'location': source[0][source[0].index('>')+1:source[0].index('<')] + ' ' + source[1][source[1].index('>')+1:source[1].index('<')]
        }
        data = json.loads(requests.get(geo_locater, params = parameters).text)['results']
        lat, lng = data[0]['locations'][0]['latLng']['lat'], data[0]['locations'][0]['latLng']['lng']
    
        temp['Address'] = parameters['location']
        temp['Geo Location'] = str( (lat,lng) )
    except:
        temp['Address'] = 'n/a'
        temp['Geo Location'] = 'n/a'

    # Acceptance Information
    try:
        source = main[main.index('card t--admission-stats'):].split('<div')
        a_rate_total = source[8][source[8].index('>')+1:source[8].index('<')]
        a_rate_w = source[17][source[17].index('mb-0')+6:source[17].index('/strong')-1]
        a_rate_m = source[20][source[20].index('mb-0')+6:source[20].index('/strong')-1]
        total_applicants = source[23][source[23].index('>')+1:source[23].index('<')]
        w_applicants = source[31][source[31].index('Women')+7:source[31].index('%')+1]
        m_applicants = source[32][source[32].index('Men')+5:source[32].index('%')+1]
        p_attend = source[39][source[39].index('>')+1:source[39].index('<')]

        temp['Acceptance Rate (T/F/M)'] = str( (a_rate_total, a_rate_w, a_rate_m) )
        temp['Total Applicants'] = total_applicants
        temp['Applicants Stats (F/M)'] = str( (w_applicants, m_applicants) )
        temp['Percent Accepted that Attend'] = p_attend
    except:
        pass
    
    

    
    colleges_dict[name] = temp


    # limit the amount of iterations
    # if i <= 1:
    #     break
    # i -= 1
    
    

In [66]:
fname = 'info5.csv' 
os.remove(fname) 
file = open(fname, 'w') 
header = 'Name' 
for key in colleges_dict[list(colleges_dict.keys())[0]]: 
    header += ',' + key 
    
file.write(header+'\n') 
    
for key in colleges_dict.keys(): 
    s = key.lower() 
    for k, val in colleges_dict[key].items(): 
        clean_val = ''.join(map(lambda x: x if x not in ',' else '', val)) 
        s += ',' + clean_val 
    file.write(s+'\n')

file.close()